# IMPORTING LIBRARIES

In [8]:
import pandas as pd
import numpy as np
import imblearn
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA


# READING CSV FILE

In [9]:
df = pd.read_csv("./CC.csv")
df.head()

,Unnamed: 0,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,2,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,3,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,4,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,5,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
X = df[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19',
          'V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']]
Y = df['Class']

# oversampling THE DATA

In [12]:
from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0) 
columns = X.columns

os_data_X,os_data_y=os.fit_sample(X,Y)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=["Class"])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of normal transcation in oversampled data",len(os_data_y[os_data_y["Class"]==0]))
print("No.of fraud transcation",len(os_data_y[os_data_y["Class"]==1]))
print("Proportion of Normal data in oversampled data is ",len(os_data_y[os_data_y["Class"]==0])/len(os_data_X))
print("Proportion of fraud data in oversampled data is ",len(os_data_y[os_data_y["Class"]==1])/len(os_data_X))

length of oversampled data is  568630
Number of normal transcation in oversampled data 284315
No.of fraud transcation 284315
Proportion of Normal data in oversampled data is  0.5
Proportion of fraud data in oversampled data is  0.5


# SPLITTING THE DATASET INTO TRAIN AND TEST

In [17]:
X = df[['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19',
          'V20','V21','V22','V23','V24','V25','V26','V27','V28','Amount']]
Y = df['Class']

X_over_train, X_over_test, y_over_train, y_over_test = train_test_split(X,Y,test_size = 0.3, random_state = 0)


# RFE ON NAIVE BAYES

In [18]:
model = BernoulliNB()
rfe = RFE(model)
fit = rfe.fit(X_over_train,y_over_train)
print("Number of Features: %d"% fit.n_features_)
print("Selected features: %s"% fit.support_)
print("Ranking of features: %s"% fit.ranking_)  

Number of Features: 14
Selected features: [ True False  True False  True  True  True False  True  True False  True
 False  True  True  True  True  True False False False False  True False
 False False False False False]
Ranking of features: [ 1 13  1 16  1  1  1 10  1  1 15  1  4  1  1  1  1  1  8  7 12  3  1  2
  6  5 11  9 14]


# MODEL FITTING ON TRAIN DATA

In [20]:
gnb = GaussianNB()
X = X_over_train[['V1','V3','V5', 'V6', 'V7','V9','V10','V12','V14','V15','V16','V17','V18','V23','V24']]
gnb.fit(X,y_over_train)
y_pred = gnb.predict(X)
print(confusion_matrix(y_over_train,y_pred))  
print(classification_report(y_over_train,y_pred))
print(accuracy_score(y_over_train, y_pred))

[[196150   2869]
 [    53    292]]
             precision    recall  f1-score   support

          0       1.00      0.99      0.99    199019
          1       0.09      0.85      0.17       345

avg / total       1.00      0.99      0.99    199364

0.9853433919865171


# MODEL FITTING ON TEST DATA

In [21]:
X2 = X_over_test[['V1','V3','V5', 'V6', 'V7','V9','V10','V12','V14','V15','V16','V17','V18','V23','V24']]
y_pred = gnb.predict(X2)
print(confusion_matrix(y_over_test,y_pred))  
print(classification_report(y_over_test,y_pred))
print(accuracy_score(y_over_test, y_pred))

[[84107  1189]
 [   22   125]]
             precision    recall  f1-score   support

          0       1.00      0.99      0.99     85296
          1       0.10      0.85      0.17       147

avg / total       1.00      0.99      0.99     85443

0.9858268085156185


## Grid Search

# K BEST ON NAIVE BAYES

In [22]:
#array = under_sample.values
test = SelectKBest(score_func=f_classif,k=10)
fit = test.fit(X_over_train, y_over_train)
print("scores_:",test.scores_)
print("pvalues_:",test.pvalues_)
print("selected index:",test.get_support(True))
print("after transform:",test.transform(X_over_train)) 

scores_: [2.26880063e+03 1.76190886e+03 7.95457546e+03 3.52636392e+03
 2.13445411e+03 3.53547206e+02 7.75982912e+03 7.19923395e+01
 1.91805309e+03 9.62797111e+03 4.70966484e+03 1.37731893e+04
 4.45629760e+00 1.95908635e+04 5.75834504e+00 7.82187445e+03
 2.39857737e+04 2.58424471e+03 2.24068169e+02 6.88135210e+01
 5.17236637e+02 3.79273066e+00 1.46290865e+00 7.94203949e+00
 4.31626564e+00 5.11319788e+00 1.59791737e+01 2.82729452e+01
 1.07570499e+01]
pvalues_: [0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 8.37670535e-079 0.00000000e+000 2.17481729e-017
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 3.47740297e-002 0.00000000e+000 1.64114335e-002 0.00000000e+000
 0.00000000e+000 0.00000000e+000 1.24914779e-050 1.08891219e-016
 2.36462346e-114 5.14770085e-002 2.26469324e-001 4.83041228e-003
 3.77509565e-002 2.37455879e-002 6.40661730e-005 1.05471321e-007
 1.03900644e-003]
selected index: [ 2  3  6  9 10 11 13 15 16 17]
after transform:

# MODEL FITTING ON TRAIN DATA

In [23]:
gnb = GaussianNB()
X = X_over_train[['V2','V3','V4','V9','V10','V11','V12','V14','V16','V17']]
gnb.fit(X,y_over_train)
y_pred = gnb.predict(X)
print(confusion_matrix(y_over_train,y_pred))  
print(classification_report(y_over_train,y_pred))
print(accuracy_score(y_over_train, y_pred))

[[197025   1994]
 [    51    294]]
             precision    recall  f1-score   support

          0       1.00      0.99      0.99    199019
          1       0.13      0.85      0.22       345

avg / total       1.00      0.99      0.99    199364

0.9897423807708513


# MODEL FITTING ON TEST DATA

In [50]:
X2 = X_under_test[['V2','V3','V4','V9','V10','V11','V12','V14','V16','V17']]
y_pred = gnb.predict(X2)
print(confusion_matrix(y_under_test,y_pred))  
print(classification_report(y_under_test,y_pred))
print(accuracy_score(y_under_test, y_pred))

[[147   2]
 [ 22 125]]
              precision    recall  f1-score   support

           0       0.87      0.99      0.92       149
           1       0.98      0.85      0.91       147

   micro avg       0.92      0.92      0.92       296
   macro avg       0.93      0.92      0.92       296
weighted avg       0.93      0.92      0.92       296

0.918918918918919


# PCA ON NAIVE BAYES

In [51]:
pca = PCA(n_components=5)
X_under_train = pca.fit_transform(X_under_train)
X_under_test = pca.transform(X_under_test)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

[9.99970246e-01 2.96341777e-05 8.05741247e-08 1.06952040e-08
 1.04275952e-08]


# MODEL FITTING ON TRAIN & TEST DATA

In [52]:
gnb = GaussianNB()
gnb.fit(X_under_train,y_under_train)
y_pred = gnb.predict(X_under_test)
print(confusion_matrix(y_under_test,y_pred))  
print(classification_report(y_under_test,y_pred))
print(accuracy_score(y_under_test, y_pred))

[[147   2]
 [ 42 105]]
              precision    recall  f1-score   support

           0       0.78      0.99      0.87       149
           1       0.98      0.71      0.83       147

   micro avg       0.85      0.85      0.85       296
   macro avg       0.88      0.85      0.85       296
weighted avg       0.88      0.85      0.85       296

0.8513513513513513


C:\Users\sayal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
